# Optimización de Hiperparámetros en Machine Learning

Este notebook explora diferentes técnicas de optimización de hiperparámetros usando el dataset del Titanic:
1. GridSearchCV
2. RandomizedSearchCV
3. Optimización Bayesiana (con scikit-optimize)
4. Optuna

## Preparación de datos y configuración inicial

In [1]:
# Importamos las bibliotecas necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cargamos y preparamos los datos del Titanic
def preparar_datos_titanic():
    # Cargamos los datos
    df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

    # Preprocesamiento básico
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

    # Codificación de variables categóricas
    df['Sex'] = df['Sex'].map({'female': 1, 'male': 0})
    embarked_dummies = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = pd.concat([df, embarked_dummies], axis=1)

    # Seleccionamos características
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
               'Embarked_C', 'Embarked_Q', 'Embarked_S']
    X = df[features]
    y = df['Survived']

    # Dividimos los datos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalamos las características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

In [3]:
# Cargamos los datos
X_train, X_test, y_train, y_test = preparar_datos_titanic()

## 1. GridSearchCV
Búsqueda exhaustiva en una cuadrícula de hiperparámetros

In [4]:
from sklearn.model_selection import GridSearchCV

# Definimos el espacio de búsqueda
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Creamos y ejecutamos GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Evaluamos los resultados
print("Mejores parámetros (Grid):", grid_search.best_params_)
print("Mejor puntuación (Grid):", grid_search.best_score_)
y_pred_grid = grid_search.predict(X_test)
print("\nInforme de clasificación (Grid):")
print(classification_report(y_test, y_pred_grid))

Mejores parámetros (Grid): {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Mejor puntuación (Grid): 0.8314192849404117

Informe de clasificación (Grid):
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       105
           1       0.84      0.72      0.77        74

    accuracy                           0.83       179
   macro avg       0.83      0.81      0.82       179
weighted avg       0.83      0.83      0.82       179



Al tener: Mejor puntuación (Grid): 0.8314192849404117 y  accuracy = 0.83 eso es bueno , misma puntuacion, lo malo es que la acurasi sea peor que la puntuacion.

## 2. RandomizedSearchCV
Búsqueda aleatoria en el espacio de hiperparámetros

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Definimos distribuciones para los parámetros
param_distributions = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 15),
    'min_samples_leaf': randint(1, 10)
}

# Creamos y ejecutamos RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf, param_distributions, n_iter=20, cv=5,   #  n_iter=20 cojeme solo 20 combinaciones aleatorias, NO me agas todas las combinaciones podibles
    scoring='accuracy', n_jobs=-1, random_state=42
)
random_search.fit(X_train, y_train)

# Evaluamos los resultados
print("Mejores parámetros (Random):", random_search.best_params_)
print("Mejor puntuación (Random):", random_search.best_score_)
y_pred_random = random_search.predict(X_test)
print("\nInforme de clasificación (Random):")
print(classification_report(y_test, y_pred_random))

Mejores parámetros (Random): {'max_depth': 28, 'min_samples_leaf': 7, 'min_samples_split': 12, 'n_estimators': 187}
Mejor puntuación (Random): 0.8286319314488327

Informe de clasificación (Random):
              precision    recall  f1-score   support

           0       0.81      0.90      0.86       105
           1       0.84      0.70      0.76        74

    accuracy                           0.82       179
   macro avg       0.83      0.80      0.81       179
weighted avg       0.82      0.82      0.82       179



## 3. Optimización Bayesiana
Usando scikit-optimize para búsqueda más eficiente

In [10]:
from skopt import BayesSearchCV
from skopt.space import Integer, Real

# Definimos el espacio de búsqueda
bayes_space = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(10, 50),
    'min_samples_split': Integer(2, 15),
    'min_samples_leaf': Integer(1, 10)
}

# Creamos y ejecutamos BayesSearchCV
bayes_search = BayesSearchCV(
    rf, bayes_space, n_iter=20, cv=5,
    scoring='accuracy', n_jobs=-1, random_state=42
)
bayes_search.fit(X_train, y_train)

# Evaluamos los resultados
print("Mejores parámetros (Bayes):", bayes_search.best_params_)
print("Mejor puntuación (Bayes):", bayes_search.best_score_)
y_pred_bayes = bayes_search.predict(X_test)
print("\nInforme de clasificación (Bayes):")
print(classification_report(y_test, y_pred_bayes))

Mejores parámetros (Bayes): OrderedDict([('max_depth', 41), ('min_samples_leaf', 6), ('min_samples_split', 5), ('n_estimators', 500)])
Mejor puntuación (Bayes): 0.8272037821333595

Informe de clasificación (Bayes):
              precision    recall  f1-score   support

           0       0.81      0.90      0.86       105
           1       0.84      0.70      0.76        74

    accuracy                           0.82       179
   macro avg       0.83      0.80      0.81       179
weighted avg       0.82      0.82      0.82       179



## 4. Optuna
Framework de optimización automática de hiperparámetros

In [12]:
import optuna

def objetivo(trial):
    # Definimos el espacio de búsqueda
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }

    # Creamos y entrenamos el modelo
    rf = RandomForestClassifier(**params, random_state=42)

    # Evaluamos usando validación cruzada
    scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()

# Creamos y ejecutamos el estudio de Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objetivo, n_trials=20)

# Evaluamos los resultados
print("Mejores parámetros (Optuna):", study.best_params)
print("Mejor puntuación (Optuna):", study.best_value)

# Entrenamos el modelo final con los mejores parámetros
rf_final = RandomForestClassifier(**study.best_params, random_state=42)
rf_final.fit(X_train, y_train)
y_pred_optuna = rf_final.predict(X_test)
print("\nInforme de clasificación (Optuna):")
print(classification_report(y_test, y_pred_optuna))

[I 2025-02-08 13:35:16,247] A new study created in memory with name: no-name-bc9ec37e-132a-466f-83b9-c590822b0f41
[I 2025-02-08 13:35:16,833] Trial 0 finished with value: 0.821609376538954 and parameters: {'n_estimators': 123, 'max_depth': 43, 'min_samples_split': 15, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.821609376538954.
[I 2025-02-08 13:35:18,476] Trial 1 finished with value: 0.8173840244262781 and parameters: {'n_estimators': 430, 'max_depth': 47, 'min_samples_split': 2, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.821609376538954.
[I 2025-02-08 13:35:20,124] Trial 2 finished with value: 0.821589677927706 and parameters: {'n_estimators': 447, 'max_depth': 39, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.821609376538954.
[I 2025-02-08 13:35:20,757] Trial 3 finished with value: 0.8272136314389836 and parameters: {'n_estimators': 195, 'max_depth': 31, 'min_samples_split': 13, 'min_samples_leaf': 6}. Best is trial 3 with value: 0

Mejores parámetros (Optuna): {'n_estimators': 100, 'max_depth': 25, 'min_samples_split': 13, 'min_samples_leaf': 5}
Mejor puntuación (Optuna): 0.8328474342558849

Informe de clasificación (Optuna):
              precision    recall  f1-score   support

           0       0.80      0.90      0.85       105
           1       0.82      0.69      0.75        74

    accuracy                           0.81       179
   macro avg       0.81      0.79      0.80       179
weighted avg       0.81      0.81      0.81       179



## Comparación de Resultados

In [13]:
# Creamos un DataFrame con los resultados
resultados = pd.DataFrame({
    'Método': ['GridSearchCV', 'RandomizedSearchCV', 'BayesSearchCV', 'Optuna'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_grid),
        accuracy_score(y_test, y_pred_random),
        accuracy_score(y_test, y_pred_bayes),
        accuracy_score(y_test, y_pred_optuna)
    ]
})

print("Comparación de resultados:")
print(resultados)

Comparación de resultados:
               Método  Accuracy
0        GridSearchCV  0.826816
1  RandomizedSearchCV  0.821229
2       BayesSearchCV  0.821229
3              Optuna  0.810056


## Conclusiones

Este notebook demuestra diferentes métodos de optimización de hiperparámetros:

1. GridSearchCV: Búsqueda exhaustiva pero computacionalmente costosa
2. RandomizedSearchCV: Más eficiente que Grid Search, buenos resultados con menos tiempo
3. Optimización Bayesiana: Búsqueda inteligente que aprende de iteraciones anteriores
4. Optuna: Framework moderno con características avanzadas y visualización

Cada método tiene sus ventajas y desventajas en términos de tiempo de computación y calidad de resultados.